##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзьей. Одна серия - один документ.

**To do:** Постройте небольшой модуль поискового движка, который сможет осуществлять поиск по коллекции документов.
На входе запрос и проиндексированная коллекция (в том виде, как посчитаете нужным), на выходе отсортированный по релевантности с запросом список документов коллекции. 

Релизуйте:
    - функцию препроцессинга данных
    - функцию индексирования данных
    - функцию метрики релевантности 
    - собственно, функцию поиска

[download_friends_corpus](https://yadi.sk/d/yVO1QV98CDibpw)

Напоминание про defaultdict: 
> В качестве multiple values словаря рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [1]:
### _check : в коллекции должно быть около 165 файлов

С помощью обратного индекса посчитайте:  


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) какой сезон был самым популярным у Чендлера? у Моники?

e) кто из главных героев статистически самый популярный? 


# Поехали!

#### Собираем поисковик:

In [1]:
import os
import re
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

#preprocessing
def preproc(t):
    t = re.sub(r'[^\w ]',' ',t)
    t = re.sub(r' +', r' ', t)
    t = t.lower()
    t = t.split(' ')
    forms = []
    for word in t: 
        word = morph.parse(word)[0].normal_form
        forms.append(word)
    forms = ' '.join(forms)
    return forms

In [2]:
#indexing
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

def corpus():
    corpus = []
    names = []
    abspath = r'C:\Users\kapru\Desktop\infosearch-master\friends'
    for root, dirs, files in os.walk(abspath):
        #print(root)
        for name in files:
            names.append(name)
            with open(root+'\\'+name, 'r', encoding='utf-8') as f:  
                text = f.read() 
            a = preproc(text)
            corpus.append(a)
    global vectorizer
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    df = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names(), index=names)
    return df

In [3]:
data = corpus()

In [4]:
len(data)

165

In [5]:
#косинусная близость
from scipy import spatial

def relevance(q, data):
    relevance_dict = {}
    for index, row in data.iterrows():
        dataSetII = row.values
        dataSetII = dataSetII.reshape(1, -1)
        result = 1 - spatial.distance.cosine(q, dataSetII)
        relevance_dict[index] = result
    return relevance_dict

In [8]:
import math

#search 
def search(data):
    q = input()
    q = preproc(q)
    q = vectorizer.transform([q]).toarray()
    rel_dict = relevance(q, data)
    counter = 1
    for ep in sorted(rel_dict.items(), key=lambda x: x[1], reverse=True):
        if math.isnan(ep[1]) or ep[1] == 0:
            break
        print(f"Result №{counter}: {ep[0]} ({ep[1]})")
        counter += 1
    return 'done'

In [12]:
search(data)

росс
Result №1: Friends - 5x01 - The One After Ross Says Rachel.ru.txt (0.16974083664332962)
Result №2: Friends - 5x20 - The One With The Ride Along.ru.txt (0.16726726921361657)
Result №3: Friends - 4x24-25 - The One With Ross's Wedding (2).ru.txt (0.1593157533198155)
Result №4: Friends - 6x21 - The One Where Ross Meets Elizabeth's Dad.ru.txt (0.14438980106573496)
Result №5: Friends - 3x17 - The One Without The Ski Trip.ru.txt (0.13247484602382664)
Result №6: Friends - 4x23-24 - The One With Ross's Wedding (1).ru.txt (0.1323111355736455)
Result №7: Friends - 2x01 - The One With Ross's New Girlfriend.ru.txt (0.12309436920156269)
Result №8: Friends - 4x15 - The One With All The Rugby.ru.txt (0.12240030443441574)
Result №9: Friends - 2x01 - The One With Ross's New Girlfriend.DVDRip.ru.txt (0.11108783268286904)
Result №10: Friends - 4x21 - The One With The Invitations.ru.txt (0.11063367300673754)
Result №11: Friends - 5x06 - The One With The Yeti.ru.txt (0.11044271940465733)
Result №12: Fr

'done'

#### a) какое слово является самым частотным

In [13]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

def most_freq(data):
    data = data.transpose()
    data['sum'] = data.sum(axis=1)
    most_freq = data.sort_values(by=['sum'], ascending=False)
    return most_freq

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kapru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
most_freq(data).head()

,Friends - 1x01 - The One Where Monica Gets A Roommate.ru.txt,Friends - 1x02 - The One With The Sonogram At The End.ru.txt,Friends - 1x03 - The One With The Thumb.ru.txt,Friends - 1x04 - The One With George Stephanopoulos.ru.txt,Friends - 1x05 - The One With The East German Laundry Detergent.ru.txt,Friends - 1x06 - The One With The Butt.ru.txt,Friends - 1x07 - The One With The Blackout.ru.txt,Friends - 1x08 - The One Where Nana Dies Twice.ru.txt,Friends - 1x09 - The One Where Underdog Gets Away.ru.txt,Friends - 1x10 - The One With The Monkey.ru.txt,...,Friends - 7x16 - The One With The Truth About London.ru.txt,Friends - 7x17 - The One With The Cheap Wedding Dress.ru.txt,Friends - 7x18 - The One With Joey's Award.ru.txt,Friends - 7x19 - The One With Ross And Monica's Cousin.ru.txt,Friends - 7x20 - The One With Rachel's Big Kiss.ru.txt,Friends - 7x21 - The One With The Vows.ru.txt,Friends - 7x22 - The One With Chandler's Dad.ru.txt,Friends - 7x23 - The One With Chandler And Monica's Wedding (1).ru.txt,Friends - 7x24-25 - The One With Chandler And Monica's Wedding (2).ru.txt,sum
ты,100,41,51,51,63,62,49,42,68,68,...,72,55,61,57,75,61,58,50,77,11193
не,77,41,43,49,50,43,44,39,43,52,...,71,48,71,54,65,70,55,61,60,9456
что,64,32,36,57,47,41,43,43,45,32,...,64,47,55,66,74,78,49,66,44,9448
это,63,31,51,31,45,47,42,25,24,33,...,48,38,43,32,59,43,33,19,34,7529
быть,44,23,26,28,28,20,18,19,24,27,...,37,22,35,23,41,18,21,22,23,4768


Самое частотное слово - **ТЫ**

#### b) какое слово является самым редким

In [15]:
# напишем функцию, которая будет находить слова, которые встретились только один раз
def least_freq(data):
    data = data.transpose()
    data['sum'] = data.sum(axis=1)
    least_freq = data.sort_values(by=['sum'], ascending=False)
    least_freq = data['sum']==1
    least_freq = data[least_freq]
    least_list = least_freq.index.values.tolist()
    result = []
    for l_l in least_list:
        if re.search(r'[0-9A-Za-z]', l_l) is None:
            result.append(l_l)
    return result

In [16]:
print (least_freq(data))

['аааа', 'ааааааа', 'аааааау', 'аба', 'абонемент', 'абрикос', 'абсурд', 'абсурдный', 'авансом', 'аварийный', 'авиа', 'авиамодельный', 'авианосец', 'авиапочта', 'аврора', 'австралийский', 'австралия', 'авто', 'автобизнес', 'автобусный', 'автомобиль', 'авторство', 'автостоп', 'агамемнон', 'агрегат', 'ад', 'адвокатский', 'аделаида', 'адидас', 'адно', 'адресат', 'адресовать', 'адский', 'азарт', 'азартный', 'академический', 'акесента', 'аккумулятор', 'аккуратничать', 'акр', 'активно', 'активность', 'акула', 'акустика', 'аланость', 'алгонкин', 'алда', 'алджернон', 'алек', 'алиментный', 'алименты', 'алкоголик', 'алкоголичка', 'алкогольный', 'аллах', 'аллен', 'аллергичный', 'алтеа', 'алый', 'альбайно', 'альвареза', 'альтернатива', 'ама', 'амандочка', 'амбициозный', 'амбиция', 'амери', 'американ', 'американск', 'амстердам', 'амузбушнуть', 'амур', 'амфибия', 'аналитик', 'аналогичный', 'анастассакиснуть', 'анатомировать', 'анатомически', 'анджелес', 'анжел', 'анжелос', 'анжелось', 'аниматор', 'ан

In [17]:
print ('Первые 30 редких слов: ', least_freq(data)[:30])

Первые 30 редких слов:  ['аааа', 'ааааааа', 'аааааау', 'аба', 'абонемент', 'абрикос', 'абсурд', 'абсурдный', 'авансом', 'аварийный', 'авиа', 'авиамодельный', 'авианосец', 'авиапочта', 'аврора', 'австралийский', 'австралия', 'авто', 'автобизнес', 'автобусный', 'автомобиль', 'авторство', 'автостоп', 'агамемнон', 'агрегат', 'ад', 'адвокатский', 'аделаида', 'адидас', 'адно']


#### c) какой набор слов есть во всех документах коллекции

In [18]:
data

,00,000,007,009,02,038,03815,0fps,10,100,...,ёвить,ёй,ёкнуть,ёлка,ёлочный,ёпэрэсотэ,ёрл,ёрш,ёршик,ёще
Friends - 1x01 - The One Where Monica Gets A Roommate.ru.txt,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x02 - The One With The Sonogram At The End.ru.txt,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x03 - The One With The Thumb.ru.txt,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x04 - The One With George Stephanopoulos.ru.txt,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x05 - The One With The East German Laundry Detergent.ru.txt,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x06 - The One With The Butt.ru.txt,5,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x07 - The One With The Blackout.ru.txt,5,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x08 - The One Where Nana Dies Twice.ru.txt,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x09 - The One Where Underdog Gets Away.ru.txt,5,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Friends - 1x10 - The One With The Monkey.ru.txt,5,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
def less_freq(data):
    data = data.transpose()
    data['min'] = data.min(axis=1)
    less_freq = data.sort_values(by=['min'], ascending=False)
    return less_freq

In [20]:
less_freq_data = less_freq(data)

In [21]:
less_freq_data=less_freq_data[less_freq_data['min']!=0]

In [22]:
less_freq_data.head()

,Friends - 1x01 - The One Where Monica Gets A Roommate.ru.txt,Friends - 1x02 - The One With The Sonogram At The End.ru.txt,Friends - 1x03 - The One With The Thumb.ru.txt,Friends - 1x04 - The One With George Stephanopoulos.ru.txt,Friends - 1x05 - The One With The East German Laundry Detergent.ru.txt,Friends - 1x06 - The One With The Butt.ru.txt,Friends - 1x07 - The One With The Blackout.ru.txt,Friends - 1x08 - The One Where Nana Dies Twice.ru.txt,Friends - 1x09 - The One Where Underdog Gets Away.ru.txt,Friends - 1x10 - The One With The Monkey.ru.txt,...,Friends - 7x16 - The One With The Truth About London.ru.txt,Friends - 7x17 - The One With The Cheap Wedding Dress.ru.txt,Friends - 7x18 - The One With Joey's Award.ru.txt,Friends - 7x19 - The One With Ross And Monica's Cousin.ru.txt,Friends - 7x20 - The One With Rachel's Big Kiss.ru.txt,Friends - 7x21 - The One With The Vows.ru.txt,Friends - 7x22 - The One With Chandler's Dad.ru.txt,Friends - 7x23 - The One With Chandler And Monica's Wedding (1).ru.txt,Friends - 7x24-25 - The One With Chandler And Monica's Wedding (2).ru.txt,min
не,77,41,43,49,50,43,44,39,43,52,...,71,48,71,54,65,70,55,61,60,35
ты,100,41,51,51,63,62,49,42,68,68,...,72,55,61,57,75,61,58,50,77,28
что,64,32,36,57,47,41,43,43,45,32,...,64,47,55,66,74,78,49,66,44,27
это,63,31,51,31,45,47,42,25,24,33,...,48,38,43,32,59,43,33,19,34,16
быть,44,23,26,28,28,20,18,19,24,27,...,37,22,35,23,41,18,21,22,23,13


#### d) какой сезон был самым популярным у Чендлера? у Моники?

In [23]:
def find_by_season(q):
    q = preproc(q)
    q = vectorizer.transform([q]).toarray()
    rel_dict = relevance(q, data)

    seasons_dict = {}
    for name, freq in rel_dict.items():
        season = name.split()[2][0]
        if season not in seasons_dict:
            seasons_dict[season] = 0
        seasons_dict[season] += freq

    return seasons_dict

In [24]:
find_by_season('Чендлер')

{'1': 0.4054513513534349,
 '2': 0.32907062312099666,
 '3': 0.4602605590288613,
 '4': 0.6315973965737751,
 '5': 0.7850288582713495,
 '6': 0.7961610624274671,
 '7': 0.8550839436716484}

У Чендлера самым популярным был **7 сезон**

In [25]:
find_by_season('Моника')

{'1': 0.40416182627732655,
 '2': 0.5297923530494582,
 '3': 0.4257012794969729,
 '4': 0.4540689829635567,
 '5': 0.7848411678191626,
 '6': 0.736843650840707,
 '7': 0.9602121445342797}

У Моники самым популярным был **7 сезон**

#### e) кто из главных героев статистически самый популярный?

In [26]:
def freq_by_all_seasons(q):
    q = preproc(q)
    q = vectorizer.transform([q]).toarray()
    rel_dict = relevance(q, data)

    freq_sum = 0
    for freq in rel_dict.values():
        freq_sum += freq
    return freq_sum

def popular_character(data):
    names = ['рэйчел','моника','фиби','джой','чендлер','росс']
    max_freq = 0
    popular = ''
    for name in names:
        freq = freq_by_all_seasons(name)
        if freq > max_freq:
            max_freq = freq
            popular = name
    return popular

In [27]:
popular_character(data)

C:\Users\kapru\AppData\Roaming\Python\Python37\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


'росс'

Самый популярный герой - **Росс**